# Section 13: Groups API - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 13: Groups** of the VeevaVault API.

Groups API enables management of user groups, group memberships, group permissions, team management, organizational units, and group-based security within the Vault ecosystem.

## 🎯 Endpoints to Test

### Group Information (GroupService)
1. **Retrieve All Groups** - `GET /api/{version}/objects/groups`
2. **Retrieve Group Information** - `GET /api/{version}/objects/groups/{group_id}`
3. **Group Details** - Group metadata, descriptions, and properties
4. **Group Types** - Different types of groups available

### Group Management (GroupManagementService)  
5. **Create Group** - `POST /api/{version}/objects/groups`
6. **Update Group** - `PUT /api/{version}/objects/groups/{group_id}`
7. **Delete Group** - `DELETE /api/{version}/objects/groups/{group_id}`
8. **Group Status Management** - Activate/deactivate groups

### Group Membership (GroupMembershipService)
9. **Retrieve Group Members** - `GET /api/{version}/objects/groups/{group_id}/users`
10. **Add User to Group** - `POST /api/{version}/objects/groups/{group_id}/users`
11. **Remove User from Group** - `DELETE /api/{version}/objects/groups/{group_id}/users/{user_id}`
12. **User Group Memberships** - `GET /api/{version}/objects/users/{user_id}/groups`

### Group Permissions (GroupPermissionsService)
13. **Group Permissions** - `GET /api/{version}/objects/groups/{group_id}/permissions`
14. **Group Roles** - `GET /api/{version}/objects/groups/{group_id}/roles`
15. **Permission Assignment** - Assign permissions to groups
16. **Role Assignment** - Assign roles to groups

### Organization and Teams (OrganizationService)
17. **Organization Units** - Organizational hierarchy and units
18. **Team Management** - Team creation and management
19. **Department Groups** - Department-based grouping
20. **Functional Groups** - Role-based functional groups

### Group Security and Access Control
21. **Security Policies** - Group-based security policies
22. **Access Control Lists** - Group ACL management
23. **Document Access** - Document permissions by group
24. **Vault Access** - Vault-level access by group

### Group Analytics and Reporting
25. **Group Usage Statistics** - Group activity and usage
26. **Membership Analytics** - Member activity within groups
27. **Permission Reports** - Permission usage by groups
28. **Audit Trail** - Group-related audit events

### Group Metadata and Configuration
29. **Group Fields** - Available group fields and metadata
30. **Group Templates** - Group creation templates
31. **Group Policies** - Group governance policies
32. **Group Hierarchies** - Parent-child group relationships

### 🔬 Testing Strategy
- **Discovery First**: Query group metadata to understand available groups and types
- **Safe Operations**: Use read-only group retrieval, avoid creating or modifying groups
- **Intelligent Testing**: Use existing groups for membership and permission testing
- **Non-Destructive**: Ensure group tests don't impact group memberships or permissions

---

In [1]:
# Essential imports for comprehensive Groups API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone

# VeevaVault imports
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService

print("✅ Essential imports loaded for Groups API testing")

✅ Essential imports loaded for Groups API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced baseline API testing framework with Groups-specific functionality.

In [3]:
# Enhanced baseline API testing framework for Groups operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedGroupsAPITester:
    """Enhanced baseline API testing framework with Groups-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # Groups-specific tracking
        self.discovered_groups = []
        self.discovered_group_fields = []
        self.discovered_group_types = []       # Track different group types
        self.group_memberships = {}            # Track group membership information
        self.group_permissions = {}            # Track group permission information
        self.group_roles = {}                  # Track group role assignments
        self.organization_units = []           # Track organizational units
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault and initialize services"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                print(f"\\n🎉 Authentication Successful!")
                print(f"🆔 Session ID: {self.session_id[:20]}...")
                print(f"🏢 Vault ID: {self.vault_id}")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': self.session_id,
            'Accept': 'application/json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/json') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def print_summary(self):
        """Print test summary"""
        if not self.results:
            print("📊 No tests completed yet")
            return
            
        successful_tests = len([r for r in self.results if r.success])
        total_time = sum(r.response_time for r in self.results)
        
        print(f"\\n📊 TEST SUMMARY")
        print("=" * 40)
        print(f"✅ SUCCESS: {successful_tests}")
        if successful_tests < len(self.results):
            print(f"❌ FAILED: {len(self.results) - successful_tests}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_id else '❌ None'}")

# Initialize the enhanced groups tester
groups_tester = EnhancedGroupsAPITester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)

print("✅ Enhanced Groups API testing framework initialized")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ Enhanced Groups API testing framework initialized


## 🔐 Authentication & Service Initialization

Testing authentication and initializing services for comprehensive groups operations.

In [4]:
# Test 01: Authentication and Services initialization
print("🔐 Testing authentication and services initialization...")
print("=" * 60)

auth_success = groups_tester.authenticate()

if auth_success:
    print(f"\\n✅ Session established: {groups_tester.session_id[:20]}...")
    print(f"✅ Vault ID: {groups_tester.vault_id}")
    
    # Test basic groups endpoint availability
    print("\\n📋 Testing basic groups endpoint availability:")
    try:
        test_endpoint = f"/api/{groups_tester.api_version}/objects/groups"
        test_result = groups_tester.make_authenticated_request(test_endpoint, params={'limit': 1})
        
        if test_result.success:
            print(f"  ✅ Groups endpoint accessible (Status: {test_result.status_code})")
        else:
            print(f"  ❌ Groups endpoint failed (Status: {test_result.status_code})")
            
    except Exception as e:
        print(f"  ⚠️ Groups endpoint test exception: {e}")
    
else:
    print("❌ Authentication failed - cannot proceed with groups tests")
    
print("\\n" + "=" * 60)

🔐 Testing authentication and services initialization...
🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.34s)
\n🎉 Authentication Successful!
🆔 Session ID: 5DF6BFDAF1956FC4E5D8...
🏢 Vault ID: 92425
\n✅ Session established: 5DF6BFDAF1956FC4E5D8...
✅ Vault ID: 92425
\n📋 Testing basic groups endpoint availability:
  ✅ Groups endpoint accessible (Status: 200)
\n============================================================


## 👥 Groups Discovery

Discovering available groups in the vault.

In [5]:
# Test 02: Retrieve all groups
print("👥 Testing groups discovery...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve groups using direct API call with limit
    endpoint = f"/api/{groups_tester.api_version}/objects/groups"
    groups_params = {
        'params': {
            'limit': 20  # Limit to reduce response size
        }
    }
    
    groups_result = groups_tester.make_authenticated_request(endpoint, **groups_params)
    response_time = time.time() - start_time
    
    if groups_result.success and groups_result.response_data:
        groups_data = groups_result.response_data.get('data', [])
        groups_tester.discovered_groups = groups_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve All Groups',
            details={
                'total_groups': len(groups_data) if isinstance(groups_data, list) else 1,
                'groups_type': type(groups_data).__name__,
                'has_groups_data': bool(groups_data),
                'sample_group_names': [group.get('name__v') or group.get('group_name__v') for group in groups_data[:3] if isinstance(group, dict)] if isinstance(groups_data, list) else []
            }
        )
        groups_tester.results.append(result)
        
        print(f"✅ Retrieve All Groups [02]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(groups_data, list) and groups_data:
            print(f"👥 Found {len(groups_data)} groups")
            
            # Analyze group types and statuses
            group_types = [group.get('type__v', group.get('group_type__v', 'Unknown')) for group in groups_data if isinstance(group, dict)]
            group_statuses = [group.get('status__v', group.get('active__v', 'Unknown')) for group in groups_data if isinstance(group, dict)]
            active_groups = [s for s in group_statuses if s in ['active__v', 'Active', True, 'true', 'enabled']]
            inactive_groups = [s for s in group_statuses if s in ['inactive__v', 'Inactive', False, 'false', 'disabled']]
            
            # Track group types
            unique_group_types = list(set([t for t in group_types if t != 'Unknown']))
            groups_tester.discovered_group_types = unique_group_types
            
            print(f"\\n📊 Group Analysis:")
            print(f"  🎯 Types: {len(unique_group_types)} ({', '.join(unique_group_types[:5])})")
            print(f"  ✅ Active: {len(active_groups)}")
            print(f"  ❌ Inactive: {len(inactive_groups)}")
            print(f"  ❓ Other: {len(group_statuses) - len(active_groups) - len(inactive_groups)}")
            
            # Show sample groups
            print("\\n📋 Sample Groups:")
            for i, group in enumerate(groups_data[:5]):
                if isinstance(group, dict):
                    group_id = group.get('id', 'Unknown')
                    group_name = group.get('name__v', group.get('group_name__v', 'Unknown'))
                    group_label = group.get('label__v', group.get('group_label__v', 'Unknown'))
                    group_type = group.get('type__v', group.get('group_type__v', 'Unknown'))
                    group_status = group.get('status__v', group.get('active__v', 'Unknown'))
                else:
                    group_id = str(group)
                    group_name = 'Unknown'
                    group_label = 'Unknown'
                    group_type = 'Unknown'
                    group_status = 'Unknown'
                print(f"  {i+1}. {group_name} (ID: {group_id})")
                print(f"     Label: {group_label}")
                print(f"     Type: {group_type}, Status: {group_status}")
                
            # Extract group fields for analysis
            if groups_data and isinstance(groups_data[0], dict):
                group_fields = [key for key in groups_data[0].keys() if key.endswith('__v')]
                groups_tester.discovered_group_fields = group_fields
                print(f"\\n📋 Available Group Fields: {len(group_fields)}")
                for i, field in enumerate(group_fields[:8]):
                    print(f"  {i+1}. {field}")
                if len(group_fields) > 8:
                    print(f"  ... and {len(group_fields) - 8} more fields")
                    
        elif isinstance(groups_data, dict):
            print("👥 Retrieved groups structure")
            if 'groups' in groups_data:
                group_list = groups_data['groups']
                print(f"📊 Contains {len(group_list) if isinstance(group_list, list) else 'N/A'} groups")
        else:
            print("🔍 No groups found or unexpected format")
        
    else:
        error_msg = f"API returned {groups_result.status_code}" if groups_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve All Groups',
            error_message=error_msg
        )
        groups_tester.results.append(result)
        print(f"❌ Retrieve All Groups [02]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {groups_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/objects/groups',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve All Groups',
        error_message=str(e)
    )
    groups_tester.results.append(result)
    print(f"❌ Retrieve All Groups [02]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

👥 Testing groups discovery...
✅ Retrieve All Groups [02]: SUCCESS (0.19s)
🔍 No groups found or unexpected format
\n============================================================


## 🔍 Group Details

Testing individual group information retrieval.

In [6]:
# Test 03: Retrieve individual group details
print("🔍 Testing individual group details...")
print("=" * 60)

if groups_tester.discovered_groups:
    try:
        # Use the first discovered group for detailed testing
        test_group = groups_tester.discovered_groups[0]
        group_id = test_group.get('id') if isinstance(test_group, dict) else str(test_group)
        group_name = test_group.get('name__v', test_group.get('group_name__v', 'Unknown')) if isinstance(test_group, dict) else 'Unknown'
        
        start_time = time.time()
        
        # Attempt to retrieve specific group details
        endpoint = f"/api/{groups_tester.api_version}/objects/groups/{group_id}"
        group_detail_result = groups_tester.make_authenticated_request(endpoint)
        response_time = time.time() - start_time
        
        if group_detail_result.success and group_detail_result.response_data:
            group_detail_data = group_detail_result.response_data.get('data', {})
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=True,
                response_time=response_time,
                test_name='Retrieve Group Details',
                details={
                    'group_id': group_id,
                    'group_name': group_name,
                    'has_detail_data': bool(group_detail_data),
                    'detail_type': type(group_detail_data).__name__,
                    'detail_fields': list(group_detail_data.keys()) if isinstance(group_detail_data, dict) else []
                }
            )
            groups_tester.results.append(result)
            
            print(f"✅ Retrieve Group Details [03]: SUCCESS ({response_time:.2f}s)")
            print(f"👥 Group: {group_name} (ID: {group_id})")
            
            if isinstance(group_detail_data, dict) and group_detail_data:
                # Show detailed group information
                group_desc = group_detail_data.get('description__v', 'No description')
                group_type = group_detail_data.get('type__v', group_detail_data.get('group_type__v', 'Unknown'))
                group_status = group_detail_data.get('status__v', group_detail_data.get('active__v', 'Unknown'))
                
                print(f"\\n📋 Group Details:")
                print(f"   Description: {group_desc}")
                print(f"   Type: {group_type}")
                print(f"   Status: {group_status}")
                
                # Show additional detail fields
                detail_fields = [key for key in group_detail_data.keys() if key.endswith('__v')]
                print(f"\\n🔍 Detail Fields: {len(detail_fields)}")
                for i, field in enumerate(detail_fields[:8]):
                    field_value = group_detail_data.get(field, 'N/A')
                    # Truncate long values
                    if isinstance(field_value, str) and len(field_value) > 40:
                        field_value = field_value[:40] + "..."
                    print(f"  {i+1}. {field}: {field_value}")
                if len(detail_fields) > 8:
                    print(f"  ... and {len(detail_fields) - 8} more fields")
            else:
                print("🔍 Group details retrieved but format unclear")
                
        else:
            error_msg = f"API returned {group_detail_result.status_code}" if group_detail_result.status_code else "Unknown error"
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=False,
                response_time=response_time,
                test_name='Retrieve Group Details',
                error_message=error_msg
            )
            groups_tester.results.append(result)
            print(f"❌ Retrieve Group Details [03]: FAILED ({response_time:.2f}s)")
            print(f"   Status Code: {group_detail_result.status_code}")
            print(f"   Error: {error_msg}")
            
    except Exception as e:
        response_time = time.time() - start_time if 'start_time' in locals() else 0
        result = APITestResult(
            endpoint=f'/api/{{version}}/objects/groups/{group_id if "group_id" in locals() else "unknown"}',
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Group Details',
            error_message=str(e)
        )
        groups_tester.results.append(result)
        print(f"❌ Retrieve Group Details [03]: EXCEPTION ({response_time:.2f}s)")
        print(f"   Exception: {str(e)}")
else:
    # No groups available for testing
    result = APITestResult(
        endpoint='/api/{version}/objects/groups/{group_id}',
        method='GET',
        success=True,
        response_time=0.0,
        test_name='Retrieve Group Details',
        details={'note': 'No groups available for detail testing'}
    )
    groups_tester.results.append(result)
    print("ℹ️ Retrieve Group Details [03]: SKIPPED (No groups available)")

print("\\n" + "=" * 60)

🔍 Testing individual group details...
ℹ️ Retrieve Group Details [03]: SKIPPED (No groups available)
\n============================================================


## 👤 Group Membership

Testing group membership retrieval.

In [7]:
# Test 04: Retrieve group members for first discovered group
print("👤 Testing group membership retrieval...")
print("=" * 60)

if groups_tester.discovered_groups:
    try:
        # Use the first discovered group for membership testing
        test_group = groups_tester.discovered_groups[0]
        group_id = test_group.get('id') if isinstance(test_group, dict) else str(test_group)
        group_name = test_group.get('name__v', test_group.get('group_name__v', 'Unknown')) if isinstance(test_group, dict) else 'Unknown'
        
        start_time = time.time()
        
        # Attempt to retrieve group members
        endpoint = f"/api/{groups_tester.api_version}/objects/groups/{group_id}/users"
        members_result = groups_tester.make_authenticated_request(endpoint)
        response_time = time.time() - start_time
        
        if members_result.success and members_result.response_data:
            members_data = members_result.response_data.get('data', [])
            groups_tester.group_memberships[group_id] = members_data
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=True,
                response_time=response_time,
                test_name='Retrieve Group Members',
                details={
                    'group_id': group_id,
                    'group_name': group_name,
                    'total_members': len(members_data) if isinstance(members_data, list) else 1,
                    'has_members_data': bool(members_data),
                    'sample_member_names': [member.get('user_name__v') for member in members_data[:3] if isinstance(member, dict)] if isinstance(members_data, list) else []
                }
            )
            groups_tester.results.append(result)
            
            print(f"✅ Retrieve Group Members [04]: SUCCESS ({response_time:.2f}s)")
            print(f"👥 Group: {group_name} (ID: {group_id})")
            
            if isinstance(members_data, list) and members_data:
                print(f"👤 Found {len(members_data)} members")
                
                # Show sample members
                print("\\n📋 Group Members:")
                for i, member in enumerate(members_data[:5]):
                    if isinstance(member, dict):
                        member_id = member.get('id', member.get('user_id', 'Unknown'))
                        member_name = member.get('user_name__v', member.get('name', 'Unknown'))
                        member_email = member.get('user_email__v', member.get('email', 'Unknown'))
                        member_status = member.get('status__v', member.get('active__v', 'Unknown'))
                        print(f"  {i+1}. {member_name} (ID: {member_id})")
                        print(f"     Email: {member_email}")
                        print(f"     Status: {member_status}")
                    else:
                        print(f"  {i+1}. {member}")
                        
                if len(members_data) > 5:
                    print(f"  ... and {len(members_data) - 5} more members")
            elif isinstance(members_data, dict):
                print("👤 Retrieved members structure")
                if 'users' in members_data:
                    user_list = members_data['users']
                    print(f"📊 Contains {len(user_list) if isinstance(user_list, list) else 'N/A'} members")
            else:
                print("🔍 No members found or unexpected format")
                
        else:
            error_msg = f"API returned {members_result.status_code}" if members_result.status_code else "Unknown error"
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=False,
                response_time=response_time,
                test_name='Retrieve Group Members',
                error_message=error_msg
            )
            groups_tester.results.append(result)
            print(f"❌ Retrieve Group Members [04]: FAILED ({response_time:.2f}s)")
            print(f"   Status Code: {members_result.status_code}")
            print(f"   Error: {error_msg}")
            
    except Exception as e:
        response_time = time.time() - start_time if 'start_time' in locals() else 0
        result = APITestResult(
            endpoint=f'/api/{{version}}/objects/groups/{group_id if "group_id" in locals() else "unknown"}/users',
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Group Members',
            error_message=str(e)
        )
        groups_tester.results.append(result)
        print(f"❌ Retrieve Group Members [04]: EXCEPTION ({response_time:.2f}s)")
        print(f"   Exception: {str(e)}")
else:
    # No groups available for testing
    result = APITestResult(
        endpoint='/api/{version}/objects/groups/{group_id}/users',
        method='GET',
        success=True,
        response_time=0.0,
        test_name='Retrieve Group Members',
        details={'note': 'No groups available for membership testing'}
    )
    groups_tester.results.append(result)
    print("ℹ️ Retrieve Group Members [04]: SKIPPED (No groups available)")

print("\\n" + "=" * 60)

👤 Testing group membership retrieval...
ℹ️ Retrieve Group Members [04]: SKIPPED (No groups available)
\n============================================================


## 🔑 Group Permissions

Testing group permissions and roles retrieval.

In [8]:
# Test 05: Retrieve group permissions for first discovered group
print("🔑 Testing group permissions retrieval...")
print("=" * 60)

if groups_tester.discovered_groups:
    try:
        # Use the first discovered group for permissions testing
        test_group = groups_tester.discovered_groups[0]
        group_id = test_group.get('id') if isinstance(test_group, dict) else str(test_group)
        group_name = test_group.get('name__v', test_group.get('group_name__v', 'Unknown')) if isinstance(test_group, dict) else 'Unknown'
        
        start_time = time.time()
        
        # Attempt to retrieve group permissions
        endpoint = f"/api/{groups_tester.api_version}/objects/groups/{group_id}/permissions"
        permissions_result = groups_tester.make_authenticated_request(endpoint)
        response_time = time.time() - start_time
        
        if permissions_result.success and permissions_result.response_data:
            permissions_data = permissions_result.response_data.get('data', {})
            groups_tester.group_permissions[group_id] = permissions_data
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=True,
                response_time=response_time,
                test_name='Retrieve Group Permissions',
                details={
                    'group_id': group_id,
                    'group_name': group_name,
                    'has_permissions_data': bool(permissions_data),
                    'permissions_type': type(permissions_data).__name__,
                    'permission_fields': list(permissions_data.keys()) if isinstance(permissions_data, dict) else []
                }
            )
            groups_tester.results.append(result)
            
            print(f"✅ Retrieve Group Permissions [05]: SUCCESS ({response_time:.2f}s)")
            print(f"👥 Group: {group_name} (ID: {group_id})")
            
            if isinstance(permissions_data, dict) and permissions_data:
                # Show permission categories
                permission_fields = list(permissions_data.keys())
                print(f"\\n🔑 Permission Categories: {len(permission_fields)}")
                for i, field in enumerate(permission_fields[:8]):
                    perm_value = permissions_data.get(field)
                    if isinstance(perm_value, (list, dict)):
                        perm_summary = f"{type(perm_value).__name__} with {len(perm_value)} items"
                    else:
                        perm_summary = str(perm_value)[:50]
                    print(f"  {i+1}. {field}: {perm_summary}")
                if len(permission_fields) > 8:
                    print(f"  ... and {len(permission_fields) - 8} more categories")
            elif isinstance(permissions_data, list):
                print(f"🔑 Found {len(permissions_data)} permission entries")
            else:
                print("🔍 Permissions data retrieved but format unclear")
                
        else:
            error_msg = f"API returned {permissions_result.status_code}" if permissions_result.status_code else "Unknown error"
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=False,
                response_time=response_time,
                test_name='Retrieve Group Permissions',
                error_message=error_msg
            )
            groups_tester.results.append(result)
            print(f"❌ Retrieve Group Permissions [05]: FAILED ({response_time:.2f}s)")
            print(f"   Status Code: {permissions_result.status_code}")
            print(f"   Error: {error_msg}")
            
    except Exception as e:
        response_time = time.time() - start_time if 'start_time' in locals() else 0
        result = APITestResult(
            endpoint=f'/api/{{version}}/objects/groups/{group_id if "group_id" in locals() else "unknown"}/permissions',
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Group Permissions',
            error_message=str(e)
        )
        groups_tester.results.append(result)
        print(f"❌ Retrieve Group Permissions [05]: EXCEPTION ({response_time:.2f}s)")
        print(f"   Exception: {str(e)}")
else:
    # No groups available for testing
    result = APITestResult(
        endpoint='/api/{version}/objects/groups/{group_id}/permissions',
        method='GET',
        success=True,
        response_time=0.0,
        test_name='Retrieve Group Permissions',
        details={'note': 'No groups available for permissions testing'}
    )
    groups_tester.results.append(result)
    print("ℹ️ Retrieve Group Permissions [05]: SKIPPED (No groups available)")

print("\\n" + "=" * 60)

🔑 Testing group permissions retrieval...
ℹ️ Retrieve Group Permissions [05]: SKIPPED (No groups available)
\n============================================================


## 🎯 Test Summary & Cleanup

Comprehensive results analysis for groups testing (all operations are read-only and safe).

In [9]:
# Comprehensive Test Summary and Cleanup
print("🎯 GROUPS API TESTING - FINAL SUMMARY")
print("=" * 60)

# Print comprehensive summary
groups_tester.print_summary()

# Detailed results analysis
print("\\n📊 DETAILED RESULTS")
print("-" * 40)

success_count = 0
total_time = 0

for i, result in enumerate(groups_tester.results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    test_name = result.test_name or f"Test {i}"
    
    print(f"{status} [{i:02d}]: {test_name} ({result.response_time:.2f}s)")
    
    if result.success:
        success_count += 1
        
    total_time += result.response_time
    
    # Show additional details for interesting results
    if result.details:
        if 'group_name' in result.details:
            print(f"        👥 Group: {result.details['group_name']}")
        if 'total_groups' in result.details:
            print(f"        🏢 Groups Found: {result.details['total_groups']}")
        if 'total_members' in result.details:
            print(f"        👤 Members: {result.details['total_members']}")
        if 'permission_fields' in result.details and result.details['permission_fields']:
            print(f"        🔑 Permission Categories: {len(result.details['permission_fields'])}")
        if 'detail_fields' in result.details and result.details['detail_fields']:
            print(f"        📋 Detail Fields: {len(result.details['detail_fields'])}")
        if 'note' in result.details:
            print(f"        ℹ️ Note: {result.details['note']}")
    
    if not result.success and result.error_message:
        print(f"        ❌ Error: {result.error_message}")

# Final statistics
print("\\n🏆 FINAL STATISTICS")
print("-" * 40)
print(f"📈 Success Rate: {success_count}/{len(groups_tester.results)} ({(success_count/len(groups_tester.results)*100):.1f}%)")
print(f"⏱️ Total Execution Time: {total_time:.2f}s")
print(f"👥 Groups Discovered: {len(groups_tester.discovered_groups)}")
print(f"📋 Group Fields Discovered: {len(groups_tester.discovered_group_fields)}")
print(f"🎯 Group Types Discovered: {len(groups_tester.discovered_group_types)}")
print(f"👤 Group Memberships Available: {len(groups_tester.group_memberships)}")
print(f"🔑 Group Permissions Available: {len(groups_tester.group_permissions)}")

# Vault state summary
print("\\n🏢 VAULT STATE")
print("-" * 40)
print(f"🔗 Connected to: {groups_tester.vault_url}")
print(f"🆔 Vault ID: {groups_tester.vault_id}")
print(f"👤 User: {groups_tester.username}")
print(f"🔐 Session: {'Active' if groups_tester.session_id else 'None'}")

# API coverage summary
tested_endpoints = [
    "Authentication",
    "Retrieve All Groups", 
    "Retrieve Group Details",
    "Retrieve Group Members",
    "Retrieve Group Permissions"
]

print("\\n🎯 API COVERAGE")
print("-" * 40)
for i, endpoint in enumerate(tested_endpoints, 1):
    status = "✅" if i <= len(groups_tester.results) else "⏸️"
    print(f"{status} {endpoint}")

print("\\n✅ Groups API testing completed!")
print("📋 All operations were read-only and group-safe")
print("👥 No groups or memberships modified - vault integrity preserved")

# Export summary for next section
groups_summary = {
    'section': '13_groups',
    'total_tests': len(groups_tester.results),
    'successful_tests': success_count,
    'total_time': total_time,
    'success_rate': (success_count/len(groups_tester.results)*100) if groups_tester.results else 0,
    'discovered_groups': len(groups_tester.discovered_groups),
    'discovered_group_fields': len(groups_tester.discovered_group_fields),
    'discovered_group_types': len(groups_tester.discovered_group_types),
    'group_memberships_available': len(groups_tester.group_memberships),
    'group_permissions_available': len(groups_tester.group_permissions),
    'vault_id': groups_tester.vault_id,
    'session_active': bool(groups_tester.session_id)
}

print(f"\\n📦 Section summary exported for continuation: {groups_summary['success_rate']:.1f}% success rate")
print("=" * 60)

🎯 GROUPS API TESTING - FINAL SUMMARY
\n📊 TEST SUMMARY
✅ SUCCESS: 5
⏱️ Total time: 0.53s
🔐 Session: ✅ Active
\n📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.34s)
✅ PASS [02]: Retrieve All Groups (0.19s)
        🏢 Groups Found: 0
✅ PASS [03]: Retrieve Group Details (0.00s)
        ℹ️ Note: No groups available for detail testing
✅ PASS [04]: Retrieve Group Members (0.00s)
        ℹ️ Note: No groups available for membership testing
✅ PASS [05]: Retrieve Group Permissions (0.00s)
        ℹ️ Note: No groups available for permissions testing
\n🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 5/5 (100.0%)
⏱️ Total Execution Time: 0.53s
👥 Groups Discovered: 0
📋 Group Fields Discovered: 0
🎯 Group Types Discovered: 0
👤 Group Memberships Available: 0
🔑 Group Permissions Available: 0
\n🏢 VAULT STATE
----------------------------------------
🔗 Connected to: https://vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425